In [2]:
"""
The actual CNN along with the checker function which takes the path of the image as a string 
and tells if the mask is worn correctly or not.
"""

import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import numpy as np



import pickle

# defining img size for the checker function
IMG_SIZE = 50

# importing the pickled data (probably will need to use a different method if the dataset is too big, 
# I'm using pickle because my dataset is relatively smaller)
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)


X = np.array(X/255.0) # Normalizing the data (could use utils.normalize() but since we're dealing with graysclaed photos, dividing by 225 also works)
y = np.array(y)

model = Sequential()

# adding 2 conv2d layers 
model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))



model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


# flattening the dataset to make it ready for the dense layer
model.add(Flatten())  


model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=5, validation_split=0.1)

Epoch 1/5
107/107 [==============================] - 75s 703ms/step - loss: 0.5103 - accuracy: 0.7457 - val_loss: 0.2106 - val_accuracy: 0.9081
Epoch 2/5
107/107 [==============================] - 79s 741ms/step - loss: 0.1715 - accuracy: 0.9331 - val_loss: 0.1427 - val_accuracy: 0.9449
Epoch 3/5
107/107 [==============================] - 81s 757ms/step - loss: 0.1043 - accuracy: 0.9600 - val_loss: 0.1029 - val_accuracy: 0.9580
Epoch 4/5
107/107 [==============================] - 78s 727ms/step - loss: 0.0800 - accuracy: 0.9693 - val_loss: 0.1069 - val_accuracy: 0.9633
Epoch 5/5
107/107 [==============================] - 80s 745ms/step - loss: 0.0677 - accuracy: 0.9763 - val_loss: 0.0824 - val_accuracy: 0.9633


In [14]:
# The checker function for telling whether the person in the photo is wearing mask correctly or not
def checker(img_path):
    check_image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    check_image_resized = cv2.resize(check_image, (IMG_SIZE, IMG_SIZE))
    check_image_ready = np.array(check_image_resized).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    
    result = model.predict(check_image_ready)
    if result[0][0] > 0.5:
        print("Mask is worn incorrectly")
    elif result[0][0] < 0.5:
        print("Mask is worn correctly")
        

In [15]:
checker("test_cor.jpg")

Mask is worn correctly


In [16]:
checker("test_incor.jpg")

Mask is worn incorrectly


In [17]:
checker("test_incor2.jpg")

Mask is worn incorrectly
